In [1]:
from rdflib import Graph, Namespace, RDF, RDFS, Literal
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Construction du graphe RDF
g = Graph()
EX = Namespace("http://example.org/elearning/")

# Classes
g.add((EX.User, RDF.type, RDFS.Class))
g.add((EX.LearningResource, RDF.type, RDFS.Class))
g.add((EX.Concept, RDF.type, RDFS.Class))

# Properties
g.add((EX.likes, RDF.type, RDF.Property))
g.add((EX.relatedTo, RDF.type, RDF.Property))
g.add((EX.subConceptOf, RDF.type, RDF.Property))
g.add((EX.hasLevel, RDF.type, RDF.Property))

# Instances - Users
u1 = EX.User1
g.add((u1, RDF.type, EX.User))

# Instances - Concepts
ml = EX.MachineLearning
dl = EX.DeepLearning
cv = EX.ComputerVision

g.add((ml, RDF.type, EX.Concept))
g.add((dl, RDF.type, EX.Concept))
g.add((cv, RDF.type, EX.Concept))

g.add((dl, EX.subConceptOf, ml))
g.add((cv, EX.subConceptOf, ml))

# Instances - Courses
c1 = EX.CourseML
c2 = EX.CourseDL
c3 = EX.CourseCV

g.add((c1, RDF.type, EX.LearningResource))
g.add((c2, RDF.type, EX.LearningResource))
g.add((c3, RDF.type, EX.LearningResource))

g.add((c1, EX.relatedTo, ml))
g.add((c2, EX.relatedTo, dl))
g.add((c3, EX.relatedTo, cv))

# Preferences utilisateur
g.add((u1, EX.likes, c1))

print("Graphe RDF construit avec succès.")
print(g.serialize(format="turtle"))

Graphe RDF construit avec succès.
@prefix ns1: <http://example.org/elearning/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ns1:Concept a rdfs:Class .

ns1:LearningResource a rdfs:Class .

ns1:User a rdfs:Class .

ns1:CourseCV a ns1:LearningResource ;
    ns1:relatedTo ns1:ComputerVision .

ns1:CourseDL a ns1:LearningResource ;
    ns1:relatedTo ns1:DeepLearning .

ns1:User1 a ns1:User ;
    ns1:likes ns1:CourseML .

ns1:hasLevel a rdf:Property .

ns1:likes a rdf:Property .

ns1:relatedTo a rdf:Property .

ns1:subConceptOf a rdf:Property .

ns1:ComputerVision a ns1:Concept ;
    ns1:subConceptOf ns1:MachineLearning .

ns1:CourseML a ns1:LearningResource ;
    ns1:relatedTo ns1:MachineLearning .

ns1:DeepLearning a ns1:Concept ;
    ns1:subConceptOf ns1:MachineLearning .

ns1:MachineLearning a ns1:Concept .




In [18]:
# Requête SPARQL (Recommandation sémantique)
query = """
PREFIX ex: <http://example.org/elearning/>

SELECT DISTINCT ?recommendedCourse ?concept
WHERE {
    ex:User1 ex:likes ?likedCourse .
    ?likedCourse ex:relatedTo ?concept .
    ?subConcept ex:subConceptOf ?concept .
    ?recommendedCourse ex:relatedTo ?subConcept .
    FILTER(?recommendedCourse != ?likedCourse)
}
"""

results = g.query(query)

print("🎯 Recommandations sémantiques pour User1 :")
for row in results:
    print(f"- {row.recommendedCourse.split('/')[-1]} (via {row.concept.split('/')[-1]})")

🎯 Recommandations sémantiques pour User1 :
- CourseDL (via MachineLearning)
- CourseCV (via MachineLearning)


In [7]:
#Baseline ML classique (TF-IDF + Cosine Similarity)
        # Données textuelles simulées des cours
courses = [
    {"id": "CourseML", "text": "introduction machine learning algorithms supervised unsupervised"},
    {"id": "CourseDL", "text": "deep learning neural networks convolutional networks"},
    {"id": "CourseCV", "text": "computer vision image processing object detection"}
]

df = pd.DataFrame(courses)

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["text"])

# Similarité cosine entre cours
similarity_matrix = cosine_similarity(tfidf_matrix)

print("Matrice de similarité cosine :")
print(similarity_matrix)

Matrice de similarité cosine :
[[1.         0.08895786 0.        ]
 [0.08895786 1.         0.        ]
 [0.         0.         1.        ]]


In [8]:
#Recommandation ML basée sur similarité
def recommend_ml(course_id, df, similarity_matrix, top_k=2):
    idx = df[df["id"] == course_id].index[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_k+1]
    
    recommendations = [df.iloc[i]["id"] for i, score in sim_scores]
    return recommendations

print("🎯 Recommandations ML pour CourseML :")
print(recommend_ml("CourseML", df, similarity_matrix))

🎯 Recommandations ML pour CourseML :
['CourseDL', 'CourseCV']


In [9]:
#Comparaison des approches
print("📊 Comparaison :")
print("- Sémantique : recommandations basées sur hiérarchie de concepts (ML → DL, CV)")
print("- ML classique : recommandations basées sur similarité de texte (TF-IDF)")

📊 Comparaison :
- Sémantique : recommandations basées sur hiérarchie de concepts (ML → DL, CV)
- ML classique : recommandations basées sur similarité de texte (TF-IDF)


In [20]:
#Tests et validation des résultats

In [21]:
#Test1- Vérifier la construction du graphe RD

print("📌 Nombre total de triplets dans le graphe :", len(g))

# Vérifier l'existence de quelques triplets clés
print("Test: User1 aime CourseML ?")
print((EX.User1, EX.likes, EX.CourseML) in g)

print("Test: DeepLearning est sous-concept de MachineLearning ?")
print((EX.DeepLearning, EX.subConceptOf, EX.MachineLearning) in g)

📌 Nombre total de triplets dans le graphe : 20
Test: User1 aime CourseML ?
True
Test: DeepLearning est sous-concept de MachineLearning ?
True


In [22]:
#Test 2 – Tester la requête SPARQL (Recommandation sémantique)

results = g.query(query)
results_list = list(results)

print("📌 Nombre de recommandations sémantiques :", len(results_list))

for row in results_list:
    print("Recommandé :", row.recommendedCourse.split("/")[-1])

📌 Nombre de recommandations sémantiques : 2
Recommandé : CourseDL
Recommandé : CourseCV


In [23]:
#Test 3 – Tester le modèle ML (baseline)

reco_ml = recommend_ml("CourseML", df, similarity_matrix)

print("📌 Recommandations ML pour CourseML :", reco_ml)

📌 Recommandations ML pour CourseML : ['CourseDL', 'CourseCV']


In [24]:
#Test 4 – Comparaison sémantique vs ML

print("📊 Comparaison automatique des approches :")
print("Sémantique :", [row.recommendedCourse.split('/')[-1] for row in results_list])
print("ML :", reco_ml)

📊 Comparaison automatique des approches :
Sémantique : ['CourseDL', 'CourseCV']
ML : ['CourseDL', 'CourseCV']


In [25]:
assert (EX.User1, EX.likes, EX.CourseML) in g, "❌ Test échoué : like manquant"
assert len(results_list) > 0, "❌ Aucune recommandation sémantique trouvée"
assert len(reco_ml) > 0, "❌ Aucune recommandation ML trouvée"

print("✅ Tous les tests sont passés avec succès")

✅ Tous les tests sont passés avec succès
